human

In [1]:
import pandas as pd

# Read in your files
csv1 = pd.read_csv('all_human_GSE33000.csv') 
csv2 = pd.read_csv('same_mouse_human_gene_mapping.csv')
csv3 = pd.read_csv('all_human_GSE33000_metadata.csv')  # must have 'Accession', 'Age_weeks', 'AD'

# Step 1: Row filtering & reordering as before
csv1['ID_REF'] = csv1['ID_REF'].astype(str)
csv2['gene_id_human'] = csv2['gene_id_human'].astype(str)
ordered_genes = csv2['gene_id_human'].tolist()
csv1_filtered = csv1[csv1['ID_REF'].isin(ordered_genes)]
csv1_filtered = csv1_filtered.set_index('ID_REF')
csv1_matched_order = csv1_filtered.reindex(ordered_genes).reset_index()

# Step 2: Map age & AD for each sample column
# Make sure Accession is string and matches columns
csv3['Accession'] = csv3['Accession'].astype(str)
age_dict = dict(zip(csv3['Accession'], csv3['Age']))
ad_dict  = dict(zip(csv3['Accession'], csv3['AD']))

# For all columns that start with GSM (samples), build age and AD rows
sample_cols = [col for col in csv1_matched_order.columns if col.startswith('GSM')]

age_row = ['Age'] + [age_dict.get(col, '') for col in sample_cols]
ad_row  = ['AD']  + [ad_dict.get(col, '')  for col in sample_cols]

# Append the rows to the DataFrame
df_plus_rows = pd.concat([
    csv1_matched_order,
    pd.DataFrame([age_row, ad_row], columns=csv1_matched_order.columns)
], ignore_index=True)

# Save to file
# df_plus_rows.to_csv('csv1_reordered_with_age_AD.tsv', sep='\t', index=False)

# Preview the result
print(df_plus_rows.tail(5))

           ID_REF GSM1423780 GSM1423781 GSM1423782 GSM1423783 GSM1423784  \
9630  10023830425   0.080426  -0.027666   0.085496  -0.109619    0.10504   
9631  10023815205   0.007106   0.065091   0.040479   0.079794  -0.013048   
9632  10025912610   0.063151  -0.118265  -0.027519  -0.250529    0.05136   
9633          Age         67         88         62         90         90   
9634           AD        yes        yes        yes        yes        yes   

     GSM1423785 GSM1423786 GSM1423787 GSM1423788  ... GSM1424237 GSM1424238  \
9630  -0.100777   0.017847  -0.204545  -0.042049  ...  -0.145379   -0.07866   
9631   0.078262   -0.04752   0.058175   0.002121  ...   0.106946   0.109338   
9632   0.089817    0.23913   0.072385  -0.058781  ...  -0.070367  -0.058349   
9633         95         77        100         72  ...         50         72   
9634        yes        yes        yes        yes  ...         no         no   

     GSM1424239 GSM1424240 GSM1424241 GSM1424242 GSM1424243 GSM14242

In [2]:
import pandas as pd
import numpy as np

# df: original DataFrame (rows = features + metadata, columns = patients)

GENE_ROWS = slice(0, 9633)      # Python slice end is exclusive; row 0..9632
AGE_ROW = 9633
LABEL_ROW = 9634

df = df_plus_rows.copy()

# 1. Split components
gene_expr = df.iloc[GENE_ROWS, :].copy()
age_series = df.iloc[AGE_ROW, :].copy()
label_series = df.iloc[LABEL_ROW, :].copy()

# 2. Ensure numeric gene expression
# Coerce all to numeric; invalid parse becomes NaN
gene_expr = gene_expr.apply(pd.to_numeric, errors='coerce')

# 3. Build a new DataFrame with samples as rows
# Transpose gene expression so rows = patients
X_genes = gene_expr.T  # shape: n_patients x n_genes
X_genes.columns = [f'gene_{i}' for i in range(X_genes.shape[1])]

# Add age
age = pd.to_numeric(age_series, errors='coerce')
X_genes['age'] = age

# Labels: map yes/no (adjust mapping if your dataset uses different terms)
y = label_series.str.strip().str.lower().map({'yes': 1, 'no': 0})

# Combine
data = X_genes.copy()
data['AD'] = y

# Drop samples with missing label or age
data = data.dropna(subset=['AD', 'age'])

print(data.shape)        # (n_patients, n_genes + 2)
print(data['AD'].value_counts())

(467, 9635)
AD
1.0    310
0.0    157
Name: count, dtype: int64


In [3]:
# Drop genes with >20% missing values
threshold = 0.2 * data.shape[0]
cols_to_keep = [c for c in data.columns if c.startswith('gene_') and data[c].isna().sum() <= threshold]
cols_to_keep += ['age', 'AD']
data = data[cols_to_keep]

# Simple imputation
from sklearn.impute import SimpleImputer
features = [c for c in data.columns if c not in ('AD')]
imputer = SimpleImputer(strategy='median')
data[features] = imputer.fit_transform(data[features])

In [25]:
from sklearn.model_selection import train_test_split

X = data.drop(columns=['AD'])
y = data['AD']

X_human_train, X_human_test, y_human_train, y_human_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

mice

In [5]:
import pandas as pd

# Read in your files
csv1 = pd.read_csv('all_mice_GSE64398.csv') 
csv2 = pd.read_csv('same_mouse_human_gene_mapping.csv')
csv3 = pd.read_csv('all_mice_GSE64398_metadata.csv')  # must have 'Accession', 'Age_weeks', 'AD'

# Step 1: Row filtering & reordering as before
csv1['ID_REF'] = csv1['ID_REF'].astype(str)
csv2['gene_id_mouse'] = csv2['gene_id_mouse'].astype(str)
ordered_genes = csv2['gene_id_mouse'].tolist()
csv1_filtered = csv1[csv1['ID_REF'].isin(ordered_genes)]
csv1_filtered = csv1_filtered.set_index('ID_REF')
csv1_matched_order = csv1_filtered.reindex(ordered_genes).reset_index()

# Step 2: Map age & AD for each sample column
# Make sure Accession is string and matches columns
csv3['Accession'] = csv3['Accession'].astype(str)
age_dict = dict(zip(csv3['Accession'], csv3['Age_weeks']))
ad_dict  = dict(zip(csv3['Accession'], csv3['AD']))

# For all columns that start with GSM (samples), build age and AD rows
sample_cols = [col for col in csv1_matched_order.columns if col.startswith('GSM')]

age_row = ['age'] + [age_dict.get(col, '') for col in sample_cols]
ad_row  = ['AD']  + [ad_dict.get(col, '')  for col in sample_cols]

# Append the rows to the DataFrame
df_plus_rows = pd.concat([
    csv1_matched_order,
    pd.DataFrame([age_row, ad_row], columns=csv1_matched_order.columns)
], ignore_index=True)

# Save to file
# df_plus_rows.to_csv('csv1_reordered_with_age_AD.tsv', sep='\t', index=False)

# Preview the result
print(df_plus_rows.tail(5))

      ID_REF GSM1570253 GSM1570254 GSM1570255 GSM1570256 GSM1570257  \
9630  ZYG11B    6.93133    7.13554    6.99881    6.92082    6.81639   
9631     ZYX    12.1803    12.4074    12.2972    12.1735    12.3489   
9632    ZZZ3    6.55961    6.58151    6.71003    6.71477    6.62435   
9633     age         32         16         32         16         72   
9634      AD        yes        yes         no        yes         no   

     GSM1570258 GSM1570259 GSM1570260 GSM1570261  ... GSM1570576 GSM1570577  \
9630    6.41424    7.05888     6.9318      6.808  ...    6.85327    6.84665   
9631    11.9116     12.059    12.1774    12.2862  ...     11.622    11.9337   
9632    6.64601    6.66812    6.57711    6.58633  ...     6.7401    6.57661   
9633         16          8          8         72  ...         72         72   
9634        yes         no        yes        yes  ...         no         no   

     GSM1570578 GSM1570579 GSM1570580 GSM1570581 GSM1570582 GSM1570583  \
9630    6.96314    6.985

In [6]:
import pandas as pd
import numpy as np

# df: original DataFrame (rows = features + metadata, columns = patients)

GENE_ROWS = slice(0, 9633)      # Python slice end is exclusive; row 0..9632
AGE_ROW = 9633
LABEL_ROW = 9634

df = df_plus_rows.copy()

# 1. Split components
gene_expr = df.iloc[GENE_ROWS, :].copy()
age_series = df.iloc[AGE_ROW, :].copy()
label_series = df.iloc[LABEL_ROW, :].copy()

# 2. Ensure numeric gene expression
# Coerce all to numeric; invalid parse becomes NaN
gene_expr = gene_expr.apply(pd.to_numeric, errors='coerce')

# 3. Build a new DataFrame with samples as rows
# Transpose gene expression so rows = patients
X_genes = gene_expr.T  # shape: n_patients x n_genes
X_genes.columns = [f'gene_{i}' for i in range(X_genes.shape[1])]

# Add age
age = pd.to_numeric(age_series, errors='coerce')
X_genes['age'] = age

# Labels: map yes/no (adjust mapping if your dataset uses different terms)
y = label_series.str.strip().str.lower().map({'yes': 1, 'no': 0})

# Combine
data = X_genes.copy()
data['AD'] = y

# Drop samples with missing label or age
data = data.dropna(subset=['AD', 'age'])

print(data.shape)        # (n_patients, n_genes + 2)
print(data['AD'].value_counts())

(333, 9635)
AD
1.0    219
0.0    114
Name: count, dtype: int64


In [7]:
# Drop genes with >20% missing values
threshold = 0.2 * data.shape[0]
cols_to_keep = [c for c in data.columns if c.startswith('gene_') and data[c].isna().sum() <= threshold]
cols_to_keep += ['age', 'AD']
data = data[cols_to_keep]

# Simple imputation
from sklearn.impute import SimpleImputer
features = [c for c in data.columns if c not in ('AD')]
imputer = SimpleImputer(strategy='median')
data[features] = imputer.fit_transform(data[features])

In [26]:
from sklearn.model_selection import train_test_split

X = data.drop(columns=['AD'])
y = data['AD']

X_mouse_train, X_mouse_test, y_mouse_train, y_mouse_test = train_test_split(
    X, y, test_size=0.15, random_state=42, stratify=y
)

In [19]:
import numpy as np
import pandas as pd

# Downsample negatives in the training set to 100
N_NEG_KEEP = 50
N_POZ_KEEP = 50

train_df = X_mouse_train.copy()
train_df['AD'] = y_mouse_train

neg_idx = train_df.index[train_df['AD'] == 0]
pos_idx = train_df.index[train_df['AD'] == 1]
print(len(pos_idx))

# n_to_keep = min(N_NEG_KEEP, len(neg_idx))
neg_keep = np.random.RandomState(42).choice(neg_idx, size=N_NEG_KEEP, replace=False)
pos_keep = np.random.RandomState(42).choice(pos_idx, size=N_POZ_KEEP, replace=False)

keep_idx = np.concatenate([pos_keep, neg_keep])
train_df_ds = train_df.loc[keep_idx].sample(frac=1, random_state=42)  # shuffle

X_mouse_train = train_df_ds.drop(columns=['AD'])
y_mouse_train = train_df_ds['AD']

print('Training label counts after downsampling:')
print(y_mouse_train.value_counts())

100
Training label counts after downsampling:
AD
0.0    50
1.0    50
Name: count, dtype: int64


In [20]:
from typing import Optional, Tuple, Union, Dict, Literal
import numpy as np

try:
    import pandas as pd
except ImportError:
    pd = None  # type: ignore

from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, balanced_accuracy_score

ArrayLike = Union[np.ndarray, "pd.DataFrame"]

def _align_columns_if_dataframe(
    X_source: ArrayLike, X_target: ArrayLike
) -> Tuple[np.ndarray, np.ndarray, Optional[list]]:
    """
    If inputs are pandas DataFrames, align X_target columns to X_source column order.
    Returns numpy arrays and the aligned feature names (if DataFrames).
    If inputs are numpy arrays, returns them unchanged and feature_names=None.
    """
    if pd is not None and isinstance(X_source, pd.DataFrame) and isinstance(X_target, pd.DataFrame):
        if set(X_source.columns) != set(X_target.columns):
            missing_in_target = list(set(X_source.columns) - set(X_target.columns))
            missing_in_source = list(set(X_target.columns) - set(X_source.columns))
            raise ValueError(
                f"Feature mismatch between source and target.\n"
                f"Missing in target: {missing_in_target[:10]}{'...' if len(missing_in_target)>10 else ''}\n"
                f"Missing in source: {missing_in_source[:10]}{'...' if len(missing_in_source)>10 else ''}"
            )
        X_target_aligned = X_target.loc[:, X_source.columns]
        feature_names = list(X_source.columns)
        return X_source.values, X_target_aligned.values, feature_names
    else:
        Xs = np.asarray(X_source)
        Xt = np.asarray(X_target)
        if Xs.shape[1] != Xt.shape[1]:
            raise ValueError(f"Feature dimension mismatch: source has {Xs.shape[1]}, target has {Xt.shape[1]}")
        return Xs, Xt, None

def _pick_solver(penalty: str, solver: Optional[str]) -> str:
    if solver is not None:
        return solver
    if penalty in ("l1", "elasticnet"):
        return "saga"
    return "lbfgs"

def _fit_variance_threshold(
    Xh: np.ndarray, Xm: np.ndarray, domain: Literal["mouse", "human", "both"] = "both"
) -> VarianceThreshold:
    var = VarianceThreshold(threshold=0.0)
    if domain == "mouse":
        var.fit(Xm)
    elif domain == "human":
        var.fit(Xh)
    else:
        # Fit on stacked data so we keep features variable in either domain
        var.fit(np.vstack([Xh, Xm]))
    return var

def _fit_scaler(
    Xh_var: np.ndarray, Xm_var: np.ndarray, domain: Literal["mouse", "human", "both"] = "mouse"
) -> StandardScaler:
    scaler = StandardScaler(with_mean=True, with_std=True)
    if domain == "mouse":
        scaler.fit(Xm_var)
    elif domain == "human":
        scaler.fit(Xh_var)
    else:
        scaler.fit(np.vstack([Xh_var, Xm_var]))
    return scaler

def train_and_finetune_logreg(
    X_human_train: ArrayLike,
    y_human_train: np.ndarray,
    X_mouse_train: ArrayLike,
    y_mouse_train: np.ndarray,
    *,
    # Regularization
    C_pretrain: float = 1.0,
    C_finetune: float = 1.0,
    penalty: str = "l2",
    l1_ratio: Optional[float] = None,  # only used if penalty="elasticnet"
    solver: Optional[str] = None,      # default chosen from penalty
    # Class weights
    class_weight_pretrain: Optional[Union[str, dict]] = None,
    class_weight_finetune: Optional[Union[str, dict]] = None,
    # Preprocessing strategy
    var_fit_domain: Literal["mouse", "human", "both"] = "both",
    scaler_fit_domain: Literal["mouse", "human", "both"] = "mouse",
    # Fine-tune mixing
    human_weight: float = 0.0,  # 0 => mouse-only fine-tune; >0 => mix some human
    # Optimization
    random_state: int = 0,
    max_iter: int = 5000,
    tol: float = 1e-4,
    # Control pretraining
    pretrain_on_human: bool = True,
) -> Dict[str, object]:
    """
    Pretrain on human, fine-tune on mouse. Preprocessing is fit with a target-centric strategy
    (by default: VarianceThreshold on both domains; StandardScaler on mouse).
    """
    # 1) Align features if DataFrames
    Xh, Xm, feature_names = _align_columns_if_dataframe(X_human_train, X_mouse_train)

    # 2) Preprocessing
    var = _fit_variance_threshold(Xh, Xm, domain=var_fit_domain)
    Xh_var = var.transform(Xh)
    Xm_var = var.transform(Xm)

    scaler = _fit_scaler(Xh_var, Xm_var, domain=scaler_fit_domain)
    Xh_z = scaler.transform(Xh_var)
    Xm_z = scaler.transform(Xm_var)

    # 3) Build classifier
    solver_final = _pick_solver(penalty, solver)
    clf = LogisticRegression(
        penalty=penalty,
        solver=solver_final,
        l1_ratio=l1_ratio if penalty == "elasticnet" else None,
        max_iter=max_iter,
        tol=tol,
        random_state=random_state,
        warm_start=True,   # we will reuse weights from pretraining
    )

    # 4) Pretrain on human (optional)
    if pretrain_on_human:
        clf.set_params(C=C_pretrain, class_weight=class_weight_pretrain)
        clf.fit(Xh_z, y_human_train)

    # 5) Fine-tune on mouse (optionally mix human with a small weight)
    clf.set_params(C=C_finetune, class_weight=class_weight_finetune)
    if human_weight > 0.0:
        X_all = np.vstack([Xh_z, Xm_z])
        y_all = np.concatenate([np.asarray(y_human_train), np.asarray(y_mouse_train)])
        sw = np.concatenate([
            np.full(len(y_human_train), float(human_weight), dtype=float),
            np.ones(len(y_mouse_train), dtype=float)
        ])
        clf.fit(X_all, y_all, sample_weight=sw)
    else:
        clf.fit(Xm_z, y_mouse_train)

    kept_mask = var.get_support()
    kept_names = None
    if feature_names is not None:
        kept_names = [name for name, keep in zip(feature_names, kept_mask) if keep]

    return {
        "var": var,
        "scaler": scaler,
        "clf": clf,
        "feature_names": feature_names,
        "kept_feature_mask": kept_mask,
        "kept_feature_names": kept_names,
    }

def transform_for_inference(preproc: Dict[str, object], X: ArrayLike) -> np.ndarray:
    var: VarianceThreshold = preproc['var']
    scaler: StandardScaler = preproc['scaler']
    feature_names = preproc.get('feature_names', None)

    if pd is not None and isinstance(X, pd.DataFrame) and feature_names is not None:
        X = X.loc[:, feature_names].values
    else:
        X = np.asarray(X)
    X_var = var.transform(X)
    X_z = scaler.transform(X_var)
    return X_z

def predict_mouse(
    preproc: Dict[str, object],
    X_mouse: ArrayLike,
    return_proba: bool = True
):
    Xz = transform_for_inference(preproc, X_mouse)
    clf: LogisticRegression = preproc['clf']
    y_pred = clf.predict(Xz)
    if return_proba:
        y_proba = clf.predict_proba(Xz)[:, 1]
        return y_pred, y_proba
    return y_pred

def find_optimal_threshold(
    y_true, y_proba, metric: Literal["balanced_accuracy", "f1"] = "balanced_accuracy"
) -> float:
    """
    Choose threshold on a validation set for either balanced accuracy or F1.
    """
    thresholds = np.linspace(0.0, 1.0, 501)
    best_thr, best_score = 0.5, -np.inf
    for t in thresholds:
        y_hat = (y_proba >= t).astype(int)
        if metric == "balanced_accuracy":
            score = balanced_accuracy_score(y_true, y_hat)
        else:
            score = f1_score(y_true, y_hat)
        if score > best_score:
            best_score, best_thr = score, t
    return best_thr

In [27]:
# Train
preproc = train_and_finetune_logreg(
    X_human_train, y_human_train,
    X_mouse_train, y_mouse_train,
    # Regularization (match your baseline if needed)
    penalty="l2", solver=None,
    C_pretrain=1.0, C_finetune=1.0,
    class_weight_pretrain=None,
    class_weight_finetune=None,
    # Preprocessing strategy
    var_fit_domain="both",   # keep features variable in either domain
    scaler_fit_domain="mouse",  # scale using mouse stats
    # Fine-tune mixing
    human_weight=0.1,        # try {0.0, 0.05, 0.1, 0.2, 0.5}
    random_state=0, max_iter=5000
)

# Evaluate on mouse
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, balanced_accuracy_score, average_precision_score
y_pred, y_proba = predict_mouse(preproc, X_mouse_test)

print(classification_report(y_mouse_test, y_pred))
print("Accuracy:", accuracy_score(y_mouse_test, y_pred))
print("Balanced accuracy:", balanced_accuracy_score(y_mouse_test, y_pred))
print("ROC AUC:", roc_auc_score(y_mouse_test, y_proba))
print("Average Precision:", average_precision_score((np.asarray(y_mouse_test)==1).astype(int), y_proba))

# Optional: threshold tuning on a mouse validation fold
# thr = find_optimal_threshold(y_mouse_valid, y_proba_valid, metric="balanced_accuracy")
# y_pred_thr = (y_proba_test >= thr).astype(int)

              precision    recall  f1-score   support

         0.0       0.61      0.65      0.63        17
         1.0       0.81      0.79      0.80        33

    accuracy                           0.74        50
   macro avg       0.71      0.72      0.71        50
weighted avg       0.74      0.74      0.74        50

Accuracy: 0.74
Balanced accuracy: 0.7174688057040999
ROC AUC: 0.8003565062388591
Average Precision: 0.8986798871732948
